In [3]:
from torchmetrics.multimodal.clip_score import CLIPScore
from functools import partial

In [2]:

metrics = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

def calculate_clip_score(images, prompts):
    images_int = (images * 255).astype("uint8")
    score = metrics(torch.from_numpy(images_int).permute(0, 3, 1, 2), prompts).detach()
    return round(float(score), 4)

# sd_clip_score = calculate_clip_score(images, prompts)
# print(f"CLIP score: {sd_clip_score}")
# # CLIP score: 35.7038

/home/reutsalman/repo/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Data set & captions


In [3]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial
import io
import urllib
from datasets import load_dataset
from datasets.utils.file_utils import get_datasets_user_agent
from matplotlib import pyplot as plt
from matplotlib import image as mpimg

USER_AGENT = get_datasets_user_agent()

In [4]:
num_threads = 20
dset = load_dataset("google-research-datasets/conceptual_captions",streaming=True)


In [5]:
def fetch_single_image(image_url,timeout=None, retries=0):
    for _ in range(retries + 1):
        try:
            request = urllib.request.Request(
                image_url,
                data=None,
                headers={"user-agent": USER_AGENT},
            )
            with urllib.request.urlopen(request, timeout=timeout) as req:
                image = PIL.Image.open(io.BytesIO(req.read()))
            break
        except Exception:
            image = None
    return image


def fetch_images(batch, num_threads, timeout=None, retries=0):
    fetch_single_image_with_args = partial(fetch_single_image, timeout=timeout, retries=retries)
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        batch["image"] = list(executor.map(fetch_single_image_with_args, batch["image_url"]))
    return batch


In [6]:
def preprocess_image(img, target_size=(224, 224)):

    # Resize image while maintaining aspect ratio
    img.thumbnail(target_size, Image.ANTIALIAS)

    # Create a new image with the desired size and a white background
    new_img = Image.new("RGB", target_size, (255, 255, 255))

    # Paste the resized image onto the new image
    new_img.paste(img, ((target_size[0] - img.size[0]) // 2, (target_size[1] - img.size[1]) // 2))

    return new_img

In [16]:
dset_iter = iter(dset['train'])

In [20]:

for i in range(3):
    item2 = next(dset_iter)
    image_url = item2['image_url']
    caption = item2['caption']
    img = fetch_single_image(image_url, timeout= 70)
    img_type = type(img)
    print(f'{img}, {img_type}')
    if(img is not None):
        img = preprocess_image(img)
        sd_clip_score = calculate_clip_score(img, caption)
        print(f"CLIP score: {sd_clip_score}")

None, <class 'NoneType'>
None, <class 'NoneType'>
None, <class 'NoneType'>
